In [2]:
import json
import os
import ast
import pandas as pd

In [39]:
#importing the yelp businesses dataset
datastr=open('yelp_academic_dataset_business.json','r')
datajson=datastr.readlines()
check=[json.loads(x) for x in datajson]
ind=range(0,len(check))
business=pd.DataFrame(check,index=ind,columns=check[0].keys())

In [40]:
len(business[business['state']=='NY'])

19

In [4]:
#importing the yelp reviews dataset
datastr=open('yelp_academic_dataset_review.json','r')
datajson=datastr.readlines()
check=[json.loads(x) for x in datajson]
ind=range(0,len(check))
reviews=pd.DataFrame(check,index=ind,columns=check[0].keys())

In [ ]:
# #importing the yelp reviews dataset
# with open('yelp_academic_dataset_review.json', 'r') as f:
#     data_b = f.readlines()
    
# dff_b = map(lambda x: x.rstrip(), data_b)

# data_json_str = "[" + ','.join(dff_b) + "]"

# reviews = pd.read_json(data_json_str)

In [41]:
#To see most frequent occurences of food categories to determine what to filter business dataset on
import collections as c
emptylist=[]
temp1=[emptylist.extend(x.split(', ')) for x in business['categories'] if x!=None]
category_count=c.Counter(emptylist)
category_count.most_common(500)

[('Restaurants', 57173),
 ('Shopping', 30231),
 ('Food', 27118),
 ('Beauty & Spas', 18967),
 ('Home Services', 18634),
 ('Health & Medical', 16157),
 ('Local Services', 12906),
 ('Automotive', 12656),
 ('Nightlife', 12438),
 ('Bars', 10853),
 ('Event Planning & Services', 9774),
 ('Active Life', 9119),
 ('Fashion', 7406),
 ('Coffee & Tea', 6936),
 ('Sandwiches', 6912),
 ('Hair Salons', 6825),
 ('Fast Food', 6812),
 ('American (Traditional)', 6659),
 ('Pizza', 6603),
 ('Home & Garden', 6020),
 ('Auto Repair', 5877),
 ('Hotels & Travel', 5808),
 ('Arts & Entertainment', 5794),
 ('Professional Services', 5620),
 ('Doctors', 5450),
 ('Real Estate', 5295),
 ('Burgers', 5126),
 ('Breakfast & Brunch', 5023),
 ('Nail Salons', 4839),
 ('Fitness & Instruction', 4559),
 ('Italian', 4550),
 ('Mexican', 4419),
 ('Specialty Food', 4304),
 ('Chinese', 4247),
 ('American (New)', 4230),
 ('Pets', 4001),
 ('Hair Removal', 3766),
 ('Bakeries', 3509),
 ('Dentists', 3436),
 ('Grocery', 3380),
 ('Skin Care'

In [45]:
#defining functions to filter
def category_filter(category):
    foodstuff=('Restaurants','Food','Nightlife', 'Bars', 'Fast Food', 'Sandwiches', 'Coffee & Tea', 'American (Traditional)', 'Pizza', 'Breakfast & Brunch', 'Bakeries', 'Burgers', 'Italian', 'Mexican', 'Chinese', 'American (New)', 'Cafes', 'Ice Cream & Frozen Yogurt', 'Food Trucks', 'Desserts', 'Seafood', 'Salad', 'Sushi Bars', 'Delis', 'Juice Bars & Smoothies', 'Steakhouses')
    if category['categories']!=None:
        temp=category['categories'].split(', ')
        checksum=sum([foodstuff.count(x) for x in temp])
        return checksum
    else:
        return 0
    
def cleanstate(input_state):
    import numpy as np
    state_list = ('AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY')
    if input_state not in state_list:
        return np.NaN
    return input_state

In [46]:
#filtering for only restaurants and only US states
business['restaurant_flag']=business.apply(category_filter,axis=1)
business['state'] = business['state'].apply(cleanstate)
filtered=business[business['restaurant_flag']>0]
filtered=filtered[filtered['state'].notnull()]
len(filtered)*100/len(business)

24.943661747784912

In [130]:
restaurant_reviews=reviews[reviews["business_id"].isin(filtered["business_id"])]

In [51]:
#creating the dataset for later use

restaurant_reviews[0:10000].to_csv('restaurant_reviews_US_1000.csv', sep=',')
filtered.to_csv('businesses_filtered.csv', sep=',')

In [64]:
#basic descriptions
#restaurant_reviews[restaurant_reviews['user_id']=='msQe1u7Z_XuqjGoqhB0J5g']
print(len(restaurant_reviews))
print(len(restaurant_reviews['business_id'].unique()))

print(len(restaurant_reviews['user_id'].unique()))

3365595
47042
951201


In [131]:
#desired fields
user_rating_count=restaurant_reviews[['user_id','business_id']].groupby(['user_id']).count()
user_rating_count=user_rating_count.rename(columns = {'business_id':'user_rating_count'})
average_rating_restaurant=restaurant_reviews[['business_id','stars']].groupby(['business_id']).mean()
average_rating_restaurant=average_rating_restaurant.rename(columns = {'stars':'average_rating_rest'})
count_rating_restaurant=restaurant_reviews[['business_id','stars']].groupby(['business_id']).count()
count_rating_restaurant=count_rating_restaurant.rename(columns = {'stars':'rest_review_count'})

In [132]:
#merging desired columns back to main reviews dataset
restaurant_reviews=restaurant_reviews.merge(user_rating_count,how='left',on='user_id')
restaurant_reviews=restaurant_reviews.merge(average_rating_restaurant,how='left',on=['business_id'])
restaurant_reviews=restaurant_reviews.merge(count_rating_restaurant,how='left',on=['business_id'])

In [137]:
#checking
restaurant_reviews[restaurant_reviews['business_id']=='lGiUZcTvuMdx28YMvqZ2vw']

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,user_rating_count,average_rating_rest,rest_review_count
22844,rr-lqoQ7SutHOyEjpM0mDQ,4NtkFFtBZPWlngpjQG8UOQ,lGiUZcTvuMdx28YMvqZ2vw,5,2017-05-11,I get a large size coffee latte with sea salt ...,0,1,1,1,4.41573,89
88131,0RWQj__uN6YsJWVNYiuu7w,M9TAXyIvnT3yup8DmHbOTQ,lGiUZcTvuMdx28YMvqZ2vw,5,2017-06-24,Literally best place for boba! The place is so...,0,0,0,1,4.41573,89
184040,CRcAlyZfNz6P7hBxiM87jQ,5eZSXCh7-utaM3SikYDQSw,lGiUZcTvuMdx28YMvqZ2vw,5,2017-05-07,What a great first visit! Grabbed a free sampl...,1,0,3,2,4.41573,89
191757,V_k8lFlRzP6qP37nrsHdIA,O95bUIUAXmfuc39bvOUfAQ,lGiUZcTvuMdx28YMvqZ2vw,5,2017-09-05,Amazing slushies and smoothies! Boba is perfec...,0,0,0,5,4.41573,89
304277,K7JT60trDtn88W3NoFM_5Q,nzeiJZM6C3_UArrWVeIFHA,lGiUZcTvuMdx28YMvqZ2vw,5,2017-10-14,HIDDEN GEM!!! Thoroughly impressed. I ordered ...,0,0,0,1,4.41573,89
459149,Q4x0qoRYWNVaMKAhpVwpIA,t7LwGCCew0LYbkpjoT_OZw,lGiUZcTvuMdx28YMvqZ2vw,5,2017-05-22,Was looking for a new boba place since the one...,0,0,0,19,4.41573,89
657250,zAVlntqlUhwKR3Nf4wKVmw,h4upoaq9MNqVcLGT94_cnA,lGiUZcTvuMdx28YMvqZ2vw,5,2017-08-16,The atmosphere is very clean and simple I love...,0,0,0,3,4.41573,89
658163,V9lnkIs2lbExCdGd4eGqdg,5BATivsj3O5cjM4dubXqvA,lGiUZcTvuMdx28YMvqZ2vw,4,2017-08-29,Everyone in the office is extremely helpful ex...,0,0,0,1,4.41573,89
669770,A4Lj2yKhhLDeYQlYv_GHJw,vsffVirjOqMx99Q6-ozTyw,lGiUZcTvuMdx28YMvqZ2vw,4,2018-06-05,I got the Hokkaido with 25% sweetness and it w...,0,0,0,4,4.41573,89
726805,KgUt0bt05fIGfnsmOU57GQ,P1g9g4oMVMp1RZ2Nzzz2yg,lGiUZcTvuMdx28YMvqZ2vw,5,2017-06-24,First time coming here after reading yelp revi...,0,0,0,2,4.41573,89


In [138]:
restaurant_reviews.to_csv('restaurant_reviews_newcols.csv', sep=',')